In [35]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as con

class Initial_Conditions:
    def __init__(init, arg1, arg2, arg3, arg4, arg5, arg6, arg7, arg8, arg9):
        init.arg1 = arg1
        init.arg2 = arg2
        init.arg3 = arg3
        init.arg4 = arg4
        init.arg5 = arg5
        init.arg6 = arg6
        init.arg7 = arg7
        init.arg8 = arg8
        init.arg9 = arg9


#functions for calculating angular acceleration
def alpha_1(g, l1, l2, m1, m2, th1, th2, om1, om2):
    a1 = (-g*(2*m1 + m2)*np.sin(th1) - m2*g*np.sin(th1 - 2*th2) - 2*np.sin(th1 - th2)*m2*((om2**2)*l2 + (om1**2)*l1*np.cos(th1-th2)))/(l1*(2*m1 + m2 - m2*np.cos(2*th1 - 2*th2)))
    return a1

def alpha_2(g, l1, l2, m1, m2, th1, th2, om1, om2):
    a2 = (2*np.sin(th1 - th2)*((om1**2)*l1*(m1 + m2) + g*(m1 + m2)*np.cos(th1) + (om2**2)*l2*m2*np.cos(th1 - th2)))/(l2*(2*m1 + m2 - m2*np.cos(2*th1 - 2*th2)))
    return a2

#runge kutta step forward function for a 2nd order ODE
def rk4_step(con, kfunc1, kfunc2, step):
    
    #calculating k1 for all values
    k1omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6, con.arg7, con.arg8, con.arg9)
    k1omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6, con.arg7, con.arg8, con.arg9)
    k1theta1 = step*con.arg8
    k1theta2 = step*con.arg9
    
    #calculating k2 for all values
    k2omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k1theta1/2), con.arg7 + (k1theta2/2), con.arg8 + (k1omega1/2), con.arg9 + (k1omega2/2))
    k2omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k1theta1/2), con.arg7 + (k1theta2/2), con.arg8 + (k1omega1/2), con.arg9 + (k1omega2/2))
    k2theta1 = step*(con.arg8 + (k1omega1/2))
    k2theta2 = step*(con.arg8 + (k1omega2/2))
    
    #calculating k3 for all values
    k3omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k2theta1/2), con.arg7 + (k2theta2/2), con.arg8 + (k2omega1/2), con.arg9 + (k2omega2/2))
    k3omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k2theta1/2), con.arg7 + (k2theta2/2), con.arg8 + (k2omega1/2), con.arg9 + (k2omega2/2))
    k3theta1 = step*(con.arg8 + (k2omega1/2))
    k3theta2 = step*(con.arg8 + (k2omega2/2))
    
    #calculating k4 for all values
    k4omega1 = step*kfunc1(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + k3theta1, con.arg7 + k3theta2, con.arg8 + k3omega1, con.arg9 + k3omega2)
    k4omega2 = step*kfunc2(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, con.arg6 + (k2theta1/2), con.arg7 + (k2theta2/2), con.arg8 + (k2omega1/2), con.arg9 + (k2omega2/2))
    k4theta1 = step*(con.arg8 + k3omega1)
    k4theta2 = step*(con.arg8 + k3omega2)
    
    theta1 = con.arg6 + (k1theta1/6) + (k2theta1/3) + (k3theta1/3) + (k4theta1/6)
    theta2 = con.arg7 + (k1theta2/6) + (k2theta2/3) + (k3theta2/3) + (k4theta2/6)
    omega1 = con.arg8 + (k1omega1/6) + (k2omega1/3) + (k3omega1/3) + (k4omega1/6)
    omega2 = con.arg9 + (k1omega2/6) + (k2omega2/3) + (k3omega2/3) + (k4omega2/6)
    return(theta1, theta2, omega1, omega2)

def DP_Solver(con, t, step):
    # Initialise the arrays to be used
    # t is an array containing each of the timepoints that we will step forward h to
    t = np.arange(0,t+(step),step)
    # n is the number of timesteps in t
    n = np.shape(t)[0]
    
    #initialising arrays which will store results
    theta1 = np.zeros(n)
    theta2 = np.zeros(n)
    omega1 = np.zeros(n)
    omega2 = np.zeros(n)
    
    #set initial conditions
    theta1[0] = con.arg6
    theta2[0] = con.arg7
    omega1[0] = con.arg8
    omega2[0] = con.arg9
    
    #for loop repeatedly steps forward in time using the 4th order runge kutta method
    for i in range(1,n):
        #class sets conditions for upcoming step
        c = Initial_Conditions(con.arg1, con.arg2, con.arg3, con.arg4, con.arg5, theta1[i-1], theta2[i-1], omega1[i-1], omega2[i-1])
        values = rk4_step(c, alpha_1, alpha_2, step)
        theta1[i] = values[0]
        theta2[i] = values[1]
        omega1[i] = values[2]
        omega2[i] = values[3]
    return(t,theta1, theta2, omega1, omega2)
    

In [44]:
c1 = Initial_Conditions(con.g, 1, 1, 1, 1, np.pi/2, np.pi/2, 0, 0)

x = DP_Solver(c1, 10, 0.01)

print(x[2])

[ 1.57079633  1.57079633  1.5699791  ... -0.52969312 -0.516366
 -0.50282005]
